In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

Use this to upload the 2 csv files

In [0]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test (1).csv
Saving heart_train.csv to heart_train (1).csv


In [489]:
sys.argv = ["",input("Train:"), input("Test:")]

Train:heart_train.csv
Test:heart_test.csv


In [490]:
def check_args():
  if(len(sys.argv)!=3):
    print("Usage python CHDModel.py <heart_train.csv> <heart_test.csv>")
    sys.exit(1)
  if sys.argv[1] == "heart_train.csv":
    if sys.argv[2] == "heart_test.csv":
      print("Successfully loaded heart_train.csv")
      print("Successfully loaded heart_test.csv")
  else:
    print("failed to upload")
    sys.exit(1)
  return 
check_args()

Successfully loaded heart_train.csv
Successfully loaded heart_test.csv


In [0]:
np.set_printoptions(precision=3, suppress=True)

In [492]:
!head {sys.argv[1]}

﻿row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12,5.73,23.11,Present,49,25.3,97.2,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
6,132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0
7,142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0
8,114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1
9,114,0,3.83,19.4,Present,49,24.86,2.49,29,0


In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

In [0]:
N_VALIDATION = 66
N_TRAIN = 396
BUFFER_SIZE = 396
BATCH_SIZE = 50
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

In [0]:
def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(sys.argv[1], select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(sys.argv[2], select_columns=SELECT_COLUMNS)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [497]:
show_batch(raw_train_data)

sbp                 : [110 166 134 160 174 138 118 146 162 136 114 103 130 120 122 116 114 170
 132 122 154 158 208 114 148 136 136 112 130 112 128 146 110 158 126 188
 124 140 146 164 154 102 134 138 142 162 136 158 144 178]
tobacco             : [ 2.35  3.   14.1   4.2   0.    6.    1.25  0.    6.94  2.2   0.1   0.03
  0.08  0.    1.7   4.28  0.    0.4   4.05  0.72  2.4   6.17  7.4   9.6
 12.2  11.2   7.5   0.41  1.72  4.2   0.    7.5   0.    1.02  0.21  0.
  0.4   8.14  1.16 13.02  5.53  0.4  10.    2.27  1.32  0.    3.15  2.6
  0.76  0.95]
ldl                 : [ 3.36  3.82  4.44  6.76  3.27  7.24  4.69  4.92  4.55  4.16  3.95  4.21
  5.59  3.57  5.28  7.02  2.63  4.11  5.15  4.04  5.63  8.12  7.41  2.51
  3.79  5.81  7.39  1.88  2.66  3.58  3.09  7.21  7.14  6.33  3.95  5.47
  3.67  4.93  2.28  6.26  3.2   3.41  3.79  6.41  7.63  5.09  4.37  7.46
 10.53  4.75]
adiposity           : [26.72 26.75 22.39 37.99 35.4  37.05 31.58 18.53 33.36 38.02 15.89 18.96
 25.42 23.22 32.23 19.99  9

In [0]:
train_batch, label_batch = next(iter(raw_train_data))
test_batch, label_batch = next(iter(raw_test_data))

In [0]:
class PackNumericFeatures(object):
  def __init__(self,names):
    self.names = names
  
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features


    return features, labels

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']
FEATURES = 9

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [501]:
show_batch(packed_train_data)

famhist             : [b'Present' b'Absent' b'Absent' b'Present' b'Present' b'Present'
 b'Present' b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Absent'
 b'Present' b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Present'
 b'Absent' b'Present' b'Present' b'Present' b'Absent' b'Present' b'Absent'
 b'Absent' b'Absent' b'Present' b'Absent' b'Present' b'Present' b'Present'
 b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Present'
 b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Absent' b'Present'
 b'Absent' b'Absent']
numeric             : [[140.     8.6    3.9   32.16  52.    28.51  11.11  64.  ]
 [130.     0.     4.89  25.98  72.    30.42  14.71  23.  ]
 [146.     0.64   4.82  28.02  60.    28.11   8.23  39.  ]
 [116.     2.38   5.67  29.01  54.    27.26  15.77  51.  ]
 [132.     9.9    4.63  27.86  46.    23.39   0.51  52.  ]
 [152.    10.1    4.71  24.65  65.    26.21  24.53  57.  ]
 [138.     1.15   5.09  27.87  61.    25.65   2.34  44.  ]
 [128.     0.4   

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [503]:
import pandas as pd
desc = pd.read_csv(sys.argv[1])[NUMERIC_FEATURES].describe()
desc


,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,138.729798,3.489571,4.683939,25.509394,52.916667,26.075909,16.856717,42.878788
std,20.755849,4.315411,2.072764,7.833019,9.731880,4.231123,23.756004,14.661599
min,101.000000,0.000000,0.980000,6.740000,20.000000,14.700000,0.000000,15.000000
25%,124.000000,0.050000,3.240000,19.955000,47.000000,23.117500,0.342500,32.000000
50%,134.000000,1.805000,4.315000,26.400000,53.000000,25.920000,7.010000,45.000000
75%,148.000000,5.325000,5.685000,31.362500,59.000000,28.502500,24.262500,55.250000
max,218.000000,25.010000,15.330000,42.490000,78.000000,46.580000,145.290000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [506]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f1ce154b7b8>, mean=array([138.73 ,   3.49 ,   4.684,  25.509,  52.917,  26.076,  16.857,
        42.879]), std=array([20.756,  4.315,  2.073,  7.833,  9.732,  4.231, 23.756, 14.662])))

In [507]:
train_batch['numeric']

<tf.Tensor: id=325832, shape=(50, 8), dtype=float32, numpy=
array([[132.  ,   0.  ,   4.82,  33.41,  62.  ,  14.7 ,   0.  ,  46.  ],
       [136.  ,   2.2 ,   4.16,  38.02,  65.  ,  37.24,   4.11,  41.  ],
       [136.  ,   0.  ,   5.  ,  27.58,  49.  ,  27.59,   1.47,  39.  ],
       [176.  ,   6.  ,   3.98,  17.2 ,  52.  ,  21.07,   4.11,  61.  ],
       [160.  ,   0.6 ,   6.94,  30.53,  36.  ,  25.68,   1.42,  64.  ],
       [154.  ,   1.8 ,   7.13,  34.04,  52.  ,  35.51,  39.36,  44.  ],
       [126.  ,   3.8 ,   3.88,  31.79,  57.  ,  30.53,   0.  ,  30.  ],
       [174.  ,   2.02,   6.57,  31.9 ,  50.  ,  28.75,  11.83,  64.  ],
       [122.  ,   4.26,   4.44,  13.04,  57.  ,  19.49,  48.99,  28.  ],
       [158.  ,   2.6 ,   7.46,  34.07,  61.  ,  29.3 ,  53.28,  62.  ],
       [116.  ,   2.38,   5.67,  29.01,  54.  ,  27.26,  15.77,  51.  ],
       [160.  ,   0.  ,   2.42,  34.46,  48.  ,  29.83,   1.03,  61.  ],
       [160.  ,   4.2 ,   6.76,  37.99,  61.  ,  32.91,   3.09, 

In [508]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[-0.324, -0.809,  0.066,  1.009,  0.933, -2.689, -0.71 ,  0.213],
       [-0.132, -0.299, -0.253,  1.597,  1.242,  2.639, -0.537, -0.128],
       [-0.132, -0.809,  0.152,  0.264, -0.402,  0.358, -0.648, -0.265],
       [ 1.796,  0.582, -0.34 , -1.061, -0.094, -1.183, -0.537,  1.236],
       [ 1.025, -0.67 ,  1.088,  0.641, -1.738, -0.094, -0.65 ,  1.441],
       [ 0.736, -0.392,  1.18 ,  1.089, -0.094,  2.23 ,  0.947,  0.076],
       [-0.613,  0.072, -0.388,  0.802,  0.42 ,  1.053, -0.71 , -0.878],
       [ 1.699, -0.341,  0.91 ,  0.816, -0.3  ,  0.632, -0.212,  1.441],
       [-0.806,  0.179, -0.118, -1.592,  0.42 , -1.557,  1.353, -1.015],
       [ 0.928, -0.206,  1.339,  1.093,  0.831,  0.762,  1.533,  1.304],
       [-1.095, -0.257,  0.476,  0.447,  0.111,  0.28 , -0.046,  0.554],
       [ 1.025, -0.809, -1.092,  1.143, -0.505,  0.887, -0.666,  1.236],
       [ 1.025,  0.165,  1.002,  1.593,  0.831,  1.615, -0.58 ,  0.759],
       [-1.481,  2.667,  0.109,  1.167, -1.224,  0.

In [0]:
CATEGORIES = {
    'famhist': ['Present', 'Absent']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [511]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='famhist', vocabulary_list=('Present', 'Absent'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [512]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(train_batch).numpy()[0])

[1. 0.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [514]:
print(preprocessing_layer(train_batch).numpy()[0])

[ 1.     0.    -0.324 -0.809  0.066  1.009  0.933 -2.689 -0.71   0.213]


In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*50,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  layers.Dense(128, activation='relu', input_shape=(FEATURES,)),
  layers.Dropout(0.5),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [623]:
print("--Train--")
model.fit(train_data, epochs=1000)

--Train--
Epoch 1/1000
8/8 [==============================] - 1s 107ms/step - loss: 0.6806 - accuracy: 0.5707
Epoch 2/1000
8/8 [==============================] - 0s 12ms/step - loss: 0.6419 - accuracy: 0.6768
Epoch 3/1000
8/8 [==============================] - 0s 12ms/step - loss: 0.6026 - accuracy: 0.6717
Epoch 4/1000
8/8 [==============================] - 0s 13ms/step - loss: 0.6098 - accuracy: 0.6540
Epoch 5/1000
8/8 [==============================] - 0s 12ms/step - loss: 0.5944 - accuracy: 0.6894
Epoch 6/1000
8/8 [==============================] - 0s 11ms/step - loss: 0.6084 - accuracy: 0.6894
Epoch 7/1000
8/8 [==============================] - 0s 11ms/step - loss: 0.5887 - accuracy: 0.6944
Epoch 8/1000
8/8 [==============================] - 0s 11ms/step - loss: 0.5400 - accuracy: 0.7273
Epoch 9/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.5799 - accuracy: 0.6919
Epoch 10/1000
8/8 [==============================] - 0s 14ms/step - loss: 0.5408 - accuracy: 0.702

In [624]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}\n'.format(test_loss, test_accuracy))

2/2 [==============================] - 0s 92ms/step - loss: 1.7114 - accuracy: 0.7121


Test Loss 1.711396872997284, Test Accuracy 0.7121211886405945



In [627]:
predictions = model.predict(test_data)

for prediction, chd in zip(predictions, list(test_data)[0][1]):
  print("Predicted Diagnostic Rate: {:.2%}".format(prediction[0]),
        " | Has CHD?: ",
        ("Yes" if bool(chd) else "No"))

Predicted Diagnostic Rate: 1.14%  | Has CHD?:  No
Predicted Diagnostic Rate: 0.31%  | Has CHD?:  No
Predicted Diagnostic Rate: 99.86%  | Has CHD?:  No
Predicted Diagnostic Rate: 44.97%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 99.78%  | Has CHD?:  No
Predicted Diagnostic Rate: 1.50%  | Has CHD?:  No
Predicted Diagnostic Rate: 0.76%  | Has CHD?:  No
Predicted Diagnostic Rate: 98.94%  | Has CHD?:  No
Predicted Diagnostic Rate: 0.00%  | Has CHD?:  No
Predicted Diagnostic Rate: 1.56%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 4.08%  | Has CHD?:  No
Predicted Diagnostic Rate: 99.78%  | Has CHD?:  No
Predicted Diagnostic Rate: 35.72%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 0.23%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 8.06%  | Has CHD?:  No
Predicted Diagnostic Rate: 0.11%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 10.08%  | Has CHD?:  No
Predicted Diagnostic Rate: 6.79%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 0.05%  | Has CHD?:  Yes
Predicted Diagnostic Rate: 0.00%  | 